### Converting NetCDF to a Table

Here the ERA5-Land soil moisture netCDF is converted to a table for eventual comparison with Wang et al. (2021)

In [1]:
import xarray as xr
import pandas as pd
import dask.dataframe as dd
import os

path2data = "/Users/tejasvi/Dropbox/Database/Hydrology/era5_land_soil_moisture/"
path2out = "/Users/tejasvi/Dropbox/Database/Hydrology/era5_land_soil_moisture/processed"

# Check if the folder exists
if not os.path.exists(path2out):
    # If the folder doesn't exist, create it
    os.makedirs(path2out)
    print(f"Folder created at: {path2out}")
else:
    # If the folder exists, do nothing
    print(f"Folder already exists at: {path2out}")


Folder already exists at: /Users/tejasvi/Dropbox/Database/Hydrology/era5_land_soil_moisture/processed


In [ ]:
# Open the NetCDF file
ds = xr.open_dataset(path2data + "output_remapped.nc")
# Print to see structure (optional)
#print(ds)

# Select specific variables to include (e.g., swvl1, swvl2)
selected_vars = ['swvl1', 'swvl2', 'swvl3', 'swvl4']

# Convert selected variables to a long-format DataFrame
df = ds[selected_vars].to_dataframe().reset_index()

# Convert to Dask DataFrame
df = dd.from_pandas(df, npartitions=4)

# Preview the DataFrame
print(df.head())
print(df.shape)

        time  depth    lat  lon  depth_2  depth_3  depth_4     swvl1  \
0 1979-01-01    0.0 -89.75  0.0      7.0     28.0    100.0  0.181976   
1 1979-01-01    0.0 -89.75  0.5      7.0     28.0    100.0  0.181229   
2 1979-01-01    0.0 -89.75  1.0      7.0     28.0    100.0  0.180481   
3 1979-01-01    0.0 -89.75  1.5      7.0     28.0    100.0  0.179733   
4 1979-01-01    0.0 -89.75  2.0      7.0     28.0    100.0  0.178970   

      swvl2     swvl3     swvl4  
0  0.177338  0.198639  0.144913  
1  0.177063  0.198654  0.144913  
2  0.176743  0.198685  0.144913  
3  0.176453  0.198700  0.144913  
4  0.176193  0.198761  0.144913  
(<dask_expr.expr.Scalar: expr=df.size() // 11, dtype=int64>, 11)


Create weighted-average SM estimates for two depths:  
- 0 to 100cm
- 0 to 289cm

In [3]:
# Define the weights
weights = [7, 21, 72, 189]

# Multiply each variable by its corresponding weight
df['weighted_swvl1'] = df['swvl1'] * weights[0]
df['weighted_swvl2'] = df['swvl2'] * weights[1]
df['weighted_swvl3'] = df['swvl3'] * weights[2]
df['weighted_swvl4'] = df['swvl4'] * weights[3]

# Calculate the weighted sum and divide by the total sum of weights
df['swvl_0_100'] = (
    df['weighted_swvl1'] + df['weighted_swvl2'] + df['weighted_swvl3']
) / sum(weights[0:3])

df['swvl_0_289'] = (
    df['weighted_swvl1'] + df['weighted_swvl2'] + df['weighted_swvl3'] + df['weighted_swvl4']
) / sum(weights)

# Drop the intermediate columns (optional)
df = df.drop(columns=['weighted_swvl1', 'weighted_swvl2', 'weighted_swvl3', 'weighted_swvl4'])

# Preview the DataFrame
print(df.head())


        time  depth    lat  lon  depth_2  depth_3  depth_4     swvl1  \
0 1979-01-01    0.0 -89.75  0.0      7.0     28.0    100.0  0.181976   
1 1979-01-01    0.0 -89.75  0.5      7.0     28.0    100.0  0.181229   
2 1979-01-01    0.0 -89.75  1.0      7.0     28.0    100.0  0.180481   
3 1979-01-01    0.0 -89.75  1.5      7.0     28.0    100.0  0.179733   
4 1979-01-01    0.0 -89.75  2.0      7.0     28.0    100.0  0.178970   

      swvl2     swvl3     swvl4  swvl_0_100  swvl_0_289  
0  0.177338  0.198639  0.144913    0.192999    0.161552  
1  0.177063  0.198654  0.144913    0.192900    0.161517  
2  0.176743  0.198685  0.144913    0.192803    0.161484  
3  0.176453  0.198700  0.144913    0.192700    0.161448  
4  0.176193  0.198761  0.144913    0.192636    0.161426  


In [4]:
#Add relevant time vars
# Ensure the date_column is of string type
df['time'] = df['time'].astype(str)

# Extract 'yearmon' (first 7 characters: 'YYYY-MM')
df['yearmon'] = df['time'].str[:7]

# Extract 'year' (first 4 characters: 'YYYY')
df['year'] = df['time'].str[:4]

# Extract 'month' (characters at position 6-7: 'MM')
df['month'] = df['time'].str[5:7]

# Select and re-order the columns as requested
selected_columns = [
    'lon', 'lat', 'yearmon', 'year', 'month', 
    'swvl_0_100', 'swvl_0_289',
    'swvl1', 'swvl2', 'swvl3', 'swvl4'
]

# Re-order the DataFrame based on the selected columns
df = df[selected_columns]

# Cast columns to consistent types before merge
df['yearmon'] = df['yearmon'].astype('string')
df['year'] = df['year'].astype('int64')
df['month'] = df['month'].astype('int64')

# Preview the updated DataFrame
print(df.head())


   lon    lat  yearmon  year  month  swvl_0_100  swvl_0_289     swvl1  \
0  0.0 -89.75  1979-01  1979      1    0.192999    0.161552  0.181976   
1  0.5 -89.75  1979-01  1979      1    0.192900    0.161517  0.181229   
2  1.0 -89.75  1979-01  1979      1    0.192803    0.161484  0.180481   
3  1.5 -89.75  1979-01  1979      1    0.192700    0.161448  0.179733   
4  2.0 -89.75  1979-01  1979      1    0.192636    0.161426  0.178970   

      swvl2     swvl3     swvl4  
0  0.177338  0.198639  0.144913  
1  0.177063  0.198654  0.144913  
2  0.176743  0.198685  0.144913  
3  0.176453  0.198700  0.144913  
4  0.176193  0.198761  0.144913  


In [ ]:
# Convert to datetime
df['time'] = df.to_datetime(df['yearmon'], format='%Y-%m')

df = df.set_index(['time', 'lat', 'lon'])
ds = df.to_xarray()

Now load the Wang et al. (2021) dataset to compare outputs.

In [5]:
path2Wang = "/Users/tejasvi/Dropbox/Database/Hydrology/Wang_2021_Soil_Moisture/processed/"
# Load the CSV into a DataFrame
df_wang = pd.read_csv(path2Wang + "sm_data_monthly_1970_2016.csv")

# Convert to Dask DataFrame
df_wang = dd.from_pandas(df_wang, npartitions=4)

# Preview the DataFrame
print(df_wang.shape)
print(df_wang.head())

(<dask_expr.expr.Scalar: expr=df.size() // 7, dtype=int64>, 7)
      lon    lat  yearmon  year  month  sm_0_100  sm_0_100_rescaled
0 -179.75  65.75  1970-01  1970      1  0.269013           0.291638
1 -179.75  65.75  1970-02  1970      2  0.268694           0.282631
2 -179.75  65.75  1970-03  1970      3  0.268707           0.283012
3 -179.75  65.75  1970-04  1970      4  0.268765           0.284649
4 -179.75  65.75  1970-05  1970      5  0.270480           0.333092


In [6]:
#Convert both the datasets to yearly
# Group by lat, lon, and year
grouped = df.groupby(['lat', 'lon', 'year'])

# Compute mean for all numeric columns
df_yearly = grouped[['swvl1', 'swvl2', 'swvl3', 'swvl4', 'swvl_0_100', 'swvl_0_289']].mean()


# Group by lat, lon, and year
# grouped_wang = df_wang.groupby(['lat', 'lon', 'year'])

# Compute mean for all numeric columns
# df_wang_yearly = grouped_wang[['sm_0_100']].mean()

print(df_yearly.head())


# # Merge the two DataFrames on columns 'col1', 'col2', 'col3'
# df_merged = dd.merge(df, df_wang, on=['lon', 'lat', 'yearmon', 'year', 'month'], how='inner')

# # Compute the result (this triggers the actual computation and brings the data into memory)
# df_merged = df_merged.compute()

# # Preview the DataFrame
# print(df_merged.shape)
# print(df_merged.head())

                 swvl_0_100  swvl_0_289     swvl1     swvl2     swvl3  \
lat    lon year                                                         
-89.75 0.0 1979    0.192999    0.161552  0.181976  0.177338  0.198639   
           1980    0.192999    0.161552  0.181976  0.177338  0.198639   
           1981    0.192999    0.161552  0.181976  0.177338  0.198639   
           1982    0.192999    0.161552  0.181976  0.177338  0.198639   
           1983    0.192999    0.161552  0.181976  0.177338  0.198639   

                    swvl4  
lat    lon year            
-89.75 0.0 1979  0.144913  
           1980  0.144913  
           1981  0.144913  
           1982  0.144913  
           1983  0.144913  
